# Best Practices for Using Runners

In [1]:
%pip uninstall -y todd_ai
%pip install --no-build-isolation --extra-index-url https://pypi.org/simple .. > /dev/null


Found existing installation: todd-ai 0.4.0
Uninstalling todd-ai-0.4.0:
  Successfully uninstalled todd-ai-0.4.0
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pathlib
import tempfile
import time
from pprint import pprint
from typing import Any, NoReturn, TypedDict

import torch
import torch.nn.functional as F
import torch.utils.data
from torch import nn

import todd
from todd.runners import Memo

[2024-02-10 18:30:31,203 82437:140704363290240][patches.py:9 todd <module>] INFO: `ipdb` is installed. Using it for debugging.
/Users/bytedance/.local/share/virtualenvs/todd-ARrcnwyq/lib/python3.11/site-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(


## Preparation

### Models

In [3]:
@todd.ModelRegistry.register_()
class RunnerModel(nn.Module):

    def __init__(self) -> None:
        super().__init__()
        self._weight = torch.nn.Parameter(torch.tensor(0.0))

    @property
    def weight(self) -> torch.nn.Parameter:
        return self._weight

    def _forward(self, x: torch.Tensor) -> torch.Tensor:
        return x * self._weight

    def forward(
        self,
        runner: todd.runners.BaseRunner,
        batch,
        memo: Memo,
        *args,
        **kwargs,
    ) -> Memo:
        log: dict[str, Any] | None = memo.get("log")
        y = self._forward(batch["x"])
        loss = F.l1_loss(y, batch["y"])
        memo["loss"] = loss
        if log is not None:
            log["batch"] = str(batch)
            log["weight"] = f"{self._weight.item():.3f}"
            log["loss"] = f"{loss:.3f}"
        return memo

### Datasets

In [4]:
class Sample(TypedDict):
    x: int
    y: int

In [5]:
@todd.DatasetRegistry.register_()
class RunnerDataset(torch.utils.data.Dataset[int]):

    def __init__(self, n: int) -> None:
        self._data = list(range(1, n + 1))

    def __len__(self) -> int:
        return len(self._data)

    def __getitem__(self, index: int) -> Sample:
        x = self._data[index]
        return Sample(x=x, y=x * 2)

In [6]:
class Batch(TypedDict):
    x: torch.Tensor
    y: torch.Tensor

## Validators

In [7]:
config = todd.Config(
    type='Validator',
    name='validator',
    dataloader=dict(batch_size=1, dataset=dict(type='RunnerDataset', n=20)),
    strategy=dict(type='BaseStrategy', model=dict(type='RunnerModel')),
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = \
        todd.RunnerRegistry.build(config, work_dir=dict(root=work_dirs))
    runner.run()

    !echo
    !tree $work_dirs


[2024-02-10 18:30:35,638 82437:140704363290240][base.py:54 todd.Validator.validator __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R



/var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmpxp2c869n
└── validator

2 directories, 0 files


In [8]:
config = todd.Config(
    type='Validator',
    name='validator',
    dataloader=dict(batch_size=1, dataset=dict(type='RunnerDataset', n=20)),
    strategy=dict(type='BaseStrategy', model=dict(type='RunnerModel')),
    callbacks=[dict(type='LogCallback', interval=5)],
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = \
        todd.RunnerRegistry.build(config, work_dir=dict(root=work_dirs))
    runner.run()

    !echo
    !tree $work_dirs


[2024-02-10 18:30:36,136 82437:140704363290240][base.py:54 todd.Validator.validator __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-02-10 18:30:36,142 82437:140704363290240][log.py:93 todd.Validator.validator after_run_iter] INFO: Iter [5/20] batch={'x': tensor([5]), 'y': tensor([10])} weight=0.000 loss=10.000
[2024-02-10 18:30:36,147 82437:140704363290240][log.py:93 todd.Validator.validator after_run_iter] INFO: Iter [10/20] batch={'x': tensor([10]), 'y': tensor([20])} weight=0.000 loss=20.000
[2024-02-10 18:30:36,151 82437:140704363290240][log.py:93 todd.Validator.validator after_run_iter] INFO: Iter [15/20] batch={'x': tensor([15]), 'y': tensor([30])} weight=0.000 loss=30.000
[2024-02-10 18:30:36,155 82437:140704363290240][log.py:93 todd.Validator.validator after_run_iter] INFO: Iter [20/20] batch={'x': tensor([20]), 'y': tensor([40])} weight=0.000 loss=40.000



/var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmpbsgc4onq
└── validator

2 directories, 0 files


## Trainers

### Iteration Based

In [9]:
config = todd.Config(
    type="IterBasedTrainer",
    name="iter_based_trainer",
    dataloader=dict(
        batch_size=2,
        shuffle=True,
        dataset=dict(type="RunnerDataset", n=10),
    ),
    strategy=dict(type="BaseStrategy", model=dict(type="RunnerModel")),
    callbacks=[dict(type="LogCallback", interval=1)],
    optimizer=dict(type="SGD", lr=0.005),
    iters=8,
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = todd.RunnerRegistry.build(
        config,
        work_dir=dict(root=work_dirs),
    )
    runner.run()

[2024-02-10 18:30:36,522 82437:140704363290240][base.py:54 todd.IterBasedTrainer.iter_based_trainer __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-02-10 18:30:36,526 82437:140704363290240][log.py:93 todd.IterBasedTrainer.iter_based_trainer after_run_iter] INFO: Iter [1/8] batch={'x': tensor([2, 1]), 'y': tensor([4, 2])} weight=0.000 loss=3.000
[2024-02-10 18:30:36,529 82437:140704363290240][log.py:93 todd.IterBasedTrainer.iter_based_trainer after_run_iter] INFO: Iter [2/8] batch={'x': tensor([7, 6]), 'y': tensor([14, 12])} weight=0.000 loss=13.000
[2024-02-10 18:30:36,532 82437:140704363290240][log.py:93 todd.IterBasedTrainer.iter_based_trainer after_run_iter] INFO: Iter [3/8] batch={'x': tensor([ 8, 10]), 'y': tensor([16, 20])} weight=0.000 loss=18.000
[2024-02-10 18:30:36,535 82437:140704363290240][log.py:93 todd.IterBasedTrainer.iter_based_trainer after_run_iter] INFO: Iter [4/8] batch={'x': tensor([5, 4]), 'y': tensor([10,  8])} weight=0.000 loss=9.000
[2024-02

### Epoch Based

In [10]:
config = todd.Config(
    type="EpochBasedTrainer",
    name="epoch_based_trainer",
    dataloader=dict(
        batch_size=2,
        shuffle=True,
        dataset=dict(type="RunnerDataset", n=10),
    ),
    strategy=dict(type="BaseStrategy", model=dict(type="RunnerModel")),
    callbacks=[dict(type="LogCallback", interval=1)],
    optimizer=dict(type="SGD", lr=0.005),
    epochs=3,
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = todd.RunnerRegistry.build(
        config,
        work_dir=dict(root=work_dirs),
    )
    runner.run()

[2024-02-10 18:30:36,567 82437:140704363290240][base.py:54 todd.EpochBasedTrainer.epoch_based_trainer __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-02-10 18:30:36,570 82437:140704363290240][log.py:99 todd.EpochBasedTrainer.epoch_based_trainer before_run_epoch] INFO: Epoch [1/3]
[2024-02-10 18:30:36,572 82437:140704363290240][log.py:93 todd.EpochBasedTrainer.epoch_based_trainer after_run_iter] INFO: Iter [1/15] batch={'x': tensor([9, 5]), 'y': tensor([18, 10])} weight=0.000 loss=14.000
[2024-02-10 18:30:36,575 82437:140704363290240][log.py:93 todd.EpochBasedTrainer.epoch_based_trainer after_run_iter] INFO: Iter [2/15] batch={'x': tensor([8, 1]), 'y': tensor([16,  2])} weight=0.000 loss=9.000
[2024-02-10 18:30:36,577 82437:140704363290240][log.py:93 todd.EpochBasedTrainer.epoch_based_trainer after_run_iter] INFO: Iter [3/15] batch={'x': tensor([2, 7]), 'y': tensor([ 4, 14])} weight=0.000 loss=9.000
[2024-02-10 18:30:36,580 82437:140704363290240][log.py:93 todd.Epoch

## Callbacks

### Log

In [11]:
config = todd.Config(
    type="Validator",
    name="log_callback",
    dataloader=dict(batch_size=1, dataset=dict(type="RunnerDataset", n=20)),
    strategy=dict(type="BaseStrategy", model=dict(type="RunnerModel")),
    callbacks=[
        dict(
            type="LogCallback",
            interval=5,
            collect_env=dict(verbose=False),
        ),
    ],
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = todd.RunnerRegistry.build(
        config,
        work_dir=dict(root=work_dirs),
    )
    runner.run()

[2024-02-10 18:30:36,809 82437:140704363290240][log.py:55 todd.Validator.log_callback init] INFO: 
platform: macOS-14.0
nvidia_smi: None
python_version: 3.11.7 (main, Dec  4 2023, 18:10:11) [Clang 15.0.0 (clang-1500.1.0.2.5)]
pytorch_version: 2.0.1
torchvision_version: 0.15.2
opencv_version: 4.7.0
todd_version: 0.4.0
cuda_home: None
git_commit_id: 419d007
git_status: 
MM todd/base/eta.py
M  todd/base/stores.py
M  todd/runners/base.py
M  todd/runners/strategies/cuda.py
M  todd/runners/trainer.py
M  todd/runners/validator.py
MM todd/utils/torch.py
 M tutorials/runners.ipynb
[2024-02-10 18:30:36,812 82437:140704363290240][base.py:54 todd.Validator.log_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-02-10 18:30:36,817 82437:140704363290240][log.py:93 todd.Validator.log_callback after_run_iter] INFO: Iter [5/20] batch={'x': tensor([5]), 'y': tensor([10])} weight=0.000 loss=10.000
[2024-02-10 18:30:36,826 82437:140704363290240][log.py:93 todd.Validator.log_callba

In [12]:
config = todd.Config(
    type='Validator',
    name='log_callback',
    dataloader=dict(batch_size=1, dataset=dict(type='RunnerDataset', n=20)),
    strategy=dict(type='BaseStrategy', model=dict(type='RunnerModel')),
    callbacks=[
        dict(
            type='LogCallback',
            interval=5,
            with_file_handler=True,
        ),
    ],
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = \
        todd.RunnerRegistry.build(config, work_dir=dict(root=work_dirs))
    runner.run()

    !echo
    !tree {work_dirs}
    !echo
    !cat {work_dirs}/log_callback/*.log


[2024-02-10 18:30:36,869 82437:140704363290240][base.py:54 todd.Validator.log_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-02-10 18:30:36,874 82437:140704363290240][log.py:93 todd.Validator.log_callback after_run_iter] INFO: Iter [5/20] batch={'x': tensor([5]), 'y': tensor([10])} weight=0.000 loss=10.000
[2024-02-10 18:30:36,877 82437:140704363290240][log.py:93 todd.Validator.log_callback after_run_iter] INFO: Iter [10/20] batch={'x': tensor([10]), 'y': tensor([20])} weight=0.000 loss=20.000
[2024-02-10 18:30:36,887 82437:140704363290240][log.py:93 todd.Validator.log_callback after_run_iter] INFO: Iter [15/20] batch={'x': tensor([15]), 'y': tensor([30])} weight=0.000 loss=30.000
[2024-02-10 18:30:36,892 82437:140704363290240][log.py:93 todd.Validator.log_callback after_run_iter] INFO: Iter [20/20] batch={'x': tensor([20]), 'y': tensor([40])} weight=0.000 loss=40.000



/var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmprtb35w81
└── log_callback
    └── 2024-02-10T18-30-36_869304-08-00.log

2 directories, 1 file

[2024-02-10 18:30:36,869 82437:140704363290240][base.py:54 todd.Validator.log_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-02-10 18:30:36,874 82437:140704363290240][log.py:93 todd.Validator.log_callback after_run_iter] INFO: Iter [5/20] batch={'x': tensor([5]), 'y': tensor([10])} weight=0.000 loss=10.000
[2024-02-10 18:30:36,877 82437:140704363290240][log.py:93 todd.Validator.log_callback after_run_iter] INFO: Iter [10/20] batch={'x': tensor([10]), 'y': tensor([20])} weight=0.000 loss=20.000
[2024-02-10 18:30:36,887 82437:140704363290240][log.py:93 todd.Validator.log_callback after_run_iter] INFO: Iter [15/20] batch={'x': tensor([15]), 'y': tensor([30])} weight=0.000 loss=30.000
[2024-02-10 18:30:36,892 82437:140704363290240][log.py:93 todd.Validator.log_callback after_run_iter] INFO: Iter [20/20] batch={'x':

In [13]:
config = todd.Config(
    type="Validator",
    name="log_callback",
    dataloader=dict(batch_size=1, dataset=dict(type="RunnerDataset", n=20)),
    strategy=dict(type="BaseStrategy", model=dict(type="RunnerModel")),
    callbacks=[
        dict(
            type="LogCallback",
            interval=5,
            eta=dict(type="AverageETA"),
        ),
    ],
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = todd.RunnerRegistry.build(
        config,
        work_dir=dict(root=work_dirs),
    )
    runner.strategy.module.register_forward_hook(
        lambda *args, **kwargs: time.sleep(0.1)
    )
    runner.run()

[2024-02-10 18:30:37,589 82437:140704363290240][base.py:54 todd.Validator.log_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-02-10 18:30:38,110 82437:140704363290240][log.py:93 todd.Validator.log_callback after_run_iter] INFO: Iter [5/20] ETA 0:00:01 batch={'x': tensor([5]), 'y': tensor([10])} weight=0.000 loss=10.000
[2024-02-10 18:30:38,621 82437:140704363290240][log.py:93 todd.Validator.log_callback after_run_iter] INFO: Iter [10/20] ETA 0:00:01 batch={'x': tensor([10]), 'y': tensor([20])} weight=0.000 loss=20.000
[2024-02-10 18:30:39,137 82437:140704363290240][log.py:93 todd.Validator.log_callback after_run_iter] INFO: Iter [15/20] ETA 0:00:00 batch={'x': tensor([15]), 'y': tensor([30])} weight=0.000 loss=30.000
[2024-02-10 18:30:39,658 82437:140704363290240][log.py:93 todd.Validator.log_callback after_run_iter] INFO: Iter [20/20] ETA 0:00:00 batch={'x': tensor([20]), 'y': tensor([40])} weight=0.000 loss=40.000


In [14]:
config = todd.Config(
    type="Validator",
    name="log_callback",
    dataloader=dict(batch_size=1, dataset=dict(type="RunnerDataset", n=20)),
    strategy=dict(type="BaseStrategy", model=dict(type="RunnerModel")),
    callbacks=[
        dict(
            type="LogCallback",
            interval=5,
            eta=dict(type="EMA_ETA", ema=dict(decay=0.2)),
        ),
    ],
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = todd.RunnerRegistry.build(
        config,
        work_dir=dict(root=work_dirs),
    )
    runner.strategy.module.register_forward_hook(
        lambda *args, **kwargs: time.sleep(0.1 * min(10, runner.iter_))
    )
    runner.run()

[2024-02-10 18:30:39,673 82437:140704363290240][base.py:54 todd.Validator.log_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-02-10 18:30:41,189 82437:140704363290240][log.py:93 todd.Validator.log_callback after_run_iter] INFO: Iter [5/20] ETA 0:00:04 batch={'x': tensor([5]), 'y': tensor([10])} weight=0.000 loss=10.000
[2024-02-10 18:30:45,208 82437:140704363290240][log.py:93 todd.Validator.log_callback after_run_iter] INFO: Iter [10/20] ETA 0:00:05 batch={'x': tensor([10]), 'y': tensor([20])} weight=0.000 loss=20.000
[2024-02-10 18:30:50,224 82437:140704363290240][log.py:93 todd.Validator.log_callback after_run_iter] INFO: Iter [15/20] ETA 0:00:03 batch={'x': tensor([15]), 'y': tensor([30])} weight=0.000 loss=30.000
[2024-02-10 18:30:55,239 82437:140704363290240][log.py:93 todd.Validator.log_callback after_run_iter] INFO: Iter [20/20] ETA 0:00:00 batch={'x': tensor([20]), 'y': tensor([40])} weight=0.000 loss=40.000


In [15]:
config = todd.Config(
    type="Validator",
    name="log_callback",
    dataloader=dict(batch_size=1, dataset=dict(type="RunnerDataset", n=20)),
    strategy=dict(type="BaseStrategy", model=dict(type="RunnerModel")),
    callbacks=[
        dict(
            type="LogCallback",
            interval=5,
            collect_env=dict(verbose=False),
            with_file_handler=True,
            eta=dict(type="AverageETA"),
        ),
    ],
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = todd.RunnerRegistry.build(
        config,
        work_dir=dict(root=work_dirs),
    )
    runner.run()

[2024-02-10 18:30:55,383 82437:140704363290240][log.py:55 todd.Validator.log_callback init] INFO: 
platform: macOS-14.0
nvidia_smi: None
python_version: 3.11.7 (main, Dec  4 2023, 18:10:11) [Clang 15.0.0 (clang-1500.1.0.2.5)]
pytorch_version: 2.0.1
torchvision_version: 0.15.2
opencv_version: 4.7.0
todd_version: 0.4.0
cuda_home: None
git_commit_id: 419d007
git_status: 
MM todd/base/eta.py
M  todd/base/stores.py
M  todd/runners/base.py
M  todd/runners/strategies/cuda.py
M  todd/runners/trainer.py
M  todd/runners/validator.py
MM todd/utils/torch.py
 M tutorials/runners.ipynb
[2024-02-10 18:30:55,385 82437:140704363290240][base.py:54 todd.Validator.log_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-02-10 18:30:55,390 82437:140704363290240][log.py:93 todd.Validator.log_callback after_run_iter] INFO: Iter [5/20] ETA 0:00:00 batch={'x': tensor([5]), 'y': tensor([10])} weight=0.000 loss=10.000
[2024-02-10 18:30:55,393 82437:140704363290240][log.py:93 todd.Validato

### Git

In [16]:
config = todd.Config(
    type="Validator",
    name="git_callback",
    dataloader=dict(batch_size=1, dataset=dict(type="RunnerDataset", n=20)),
    strategy=dict(type="BaseStrategy", model=dict(type="RunnerModel")),
    callbacks=[
        dict(type="GitCallback", diff='HEAD -- ":(exclude)*.ipynb"'),
    ],
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = todd.RunnerRegistry.build(
        config,
        work_dir=dict(root=work_dirs),
    )

    !echo
    !cat {work_dirs}/git_callback/*.log

[2024-02-10 18:30:55,474 82437:140704363290240][git.py:41 todd.Validator.git_callback init] INFO: Saving git diff to /var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmpw64zoo15/git_callback/git_diff_2024-02-10T18-30-55_474606-08-00.log
[2024-02-10 18:30:55,477 82437:140704363290240][base.py:54 todd.Validator.git_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R



diff --git a/todd/base/eta.py b/todd/base/eta.py
index d4c53ea..e6bed85 100644
--- a/todd/base/eta.py
+++ b/todd/base/eta.py
@@ -9,6 +9,8 @@ import datetime
 from abc import ABC, abstractmethod
 from typing import NamedTuple
 
+from ..utils import ExponentialMovingAverage as EMA
+from .configs import Config
 from .registries import ETARegistry
 
 
@@ -28,19 +30,19 @@ class BaseETA(ABC):
         return Datum(x, t)
 
     @abstractmethod
-    def _pace(self, datum: Datum) -> float:
+    def pace(self, datum: Datum) -> float:
         pass
 
     def __call__(self, x: int) -> float:
         datum = self._datum(x)
-        pace = self._pace(datum)
+        pace = self.pace(datum)
         return pace * (self._end - x) / 1000
 
 
 @ETARegistry.register_()
 class AverageETA(BaseETA):
 
-    def _pace(self, datum: Datum) -> float:
+    def pace(self, datum: Datum) -> float:
         t = datum.t - self._start.t
         x = datum.x - self._start.x
         return t.total_seconds() * 1000 / 

### Optimize

In [17]:
config = todd.Config(
    type="IterBasedTrainer",
    name="optimize_callback",
    dataloader=dict(
        batch_size=2,
        shuffle=True,
        dataset=dict(type="RunnerDataset", n=10),
    ),
    strategy=dict(type="BaseStrategy", model=dict(type="RunnerModel")),
    callbacks=[
        dict(type='OptimizeCallback'),
        dict(type='LogCallback', interval=1),
    ],
    optimizer=dict(type="SGD", lr=0.005),
    iters=8,
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = \
        todd.RunnerRegistry.build(config, work_dir=dict(root=work_dirs))
    runner.run()

[2024-02-10 18:30:55,787 82437:140704363290240][base.py:54 todd.IterBasedTrainer.optimize_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-02-10 18:30:55,791 82437:140704363290240][log.py:93 todd.IterBasedTrainer.optimize_callback after_run_iter] INFO: Iter [1/8] batch={'x': tensor([10,  8]), 'y': tensor([20, 16])} weight=0.000 loss=18.000
[2024-02-10 18:30:55,793 82437:140704363290240][log.py:93 todd.IterBasedTrainer.optimize_callback after_run_iter] INFO: Iter [2/8] batch={'x': tensor([5, 7]), 'y': tensor([10, 14])} weight=0.045 loss=11.730
[2024-02-10 18:30:55,795 82437:140704363290240][log.py:93 todd.IterBasedTrainer.optimize_callback after_run_iter] INFO: Iter [3/8] batch={'x': tensor([9, 4]), 'y': tensor([18,  8])} weight=0.075 loss=12.513
[2024-02-10 18:30:55,798 82437:140704363290240][log.py:93 todd.IterBasedTrainer.optimize_callback after_run_iter] INFO: Iter [4/8] batch={'x': tensor([1, 3]), 'y': tensor([2, 6])} weight=0.107 loss=3.785
[2024-02-10 

### Learning Rate Schedule

In [18]:
config = todd.Config(
    type="IterBasedTrainer",
    name="lr_schedule_callback",
    dataloader=dict(
        batch_size=2,
        shuffle=True,
        dataset=dict(type="RunnerDataset", n=10),
    ),
    strategy=dict(type="BaseStrategy", model=dict(type="RunnerModel")),
    callbacks=[
        dict(type="OptimizeCallback"),
        dict(
            type="LRScheduleCallback",
            lr_scheduler=dict(type="LinearLR", total_iters=5),
        ),
        dict(type="LogCallback", interval=1),
    ],
    optimizer=dict(type="SGD", lr=0.005),
    iters=8,
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = todd.RunnerRegistry.build(
        config,
        work_dir=dict(root=work_dirs),
    )
    runner.run()

[2024-02-10 18:30:55,819 82437:140704363290240][base.py:54 todd.IterBasedTrainer.lr_schedule_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-02-10 18:30:55,822 82437:140704363290240][log.py:93 todd.IterBasedTrainer.lr_schedule_callback after_run_iter] INFO: Iter [1/8] batch={'x': tensor([7, 6]), 'y': tensor([14, 12])} weight=0.000 loss=13.000 lr=['1.667e-03']
[2024-02-10 18:30:55,824 82437:140704363290240][log.py:93 todd.IterBasedTrainer.lr_schedule_callback after_run_iter] INFO: Iter [2/8] batch={'x': tensor([8, 5]), 'y': tensor([16, 10])} weight=0.011 loss=12.930 lr=['2.333e-03']
[2024-02-10 18:30:55,826 82437:140704363290240][log.py:93 todd.IterBasedTrainer.lr_schedule_callback after_run_iter] INFO: Iter [3/8] batch={'x': tensor([4, 2]), 'y': tensor([8, 4])} weight=0.026 loss=5.922 lr=['3.000e-03']
[2024-02-10 18:30:55,828 82437:140704363290240][log.py:93 todd.IterBasedTrainer.lr_schedule_callback after_run_iter] INFO: Iter [4/8] batch={'x': tensor([10, 

In [19]:
config = todd.Config(
    type="EpochBasedTrainer",
    name="lr_schedule_callback",
    dataloader=dict(
        batch_size=2,
        shuffle=True,
        dataset=dict(type="RunnerDataset", n=4),
    ),
    strategy=dict(type="BaseStrategy", model=dict(type="RunnerModel")),
    callbacks=[
        dict(type="OptimizeCallback"),
        dict(
            type="LRScheduleCallback",
            lr_scheduler=dict(type="LinearLR", total_iters=3),
            by_epoch=True,
        ),
        dict(type="LogCallback", interval=1),
    ],
    optimizer=dict(type="SGD", lr=0.005),
    epochs=5,
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = todd.RunnerRegistry.build(
        config,
        work_dir=dict(root=work_dirs),
    )
    runner.run()

[2024-02-10 18:30:55,852 82437:140704363290240][base.py:54 todd.EpochBasedTrainer.lr_schedule_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-02-10 18:30:55,853 82437:140704363290240][log.py:99 todd.EpochBasedTrainer.lr_schedule_callback before_run_epoch] INFO: Epoch [1/5]
[2024-02-10 18:30:55,856 82437:140704363290240][log.py:93 todd.EpochBasedTrainer.lr_schedule_callback after_run_iter] INFO: Iter [1/10] batch={'x': tensor([3, 2]), 'y': tensor([6, 4])} weight=0.000 loss=5.000 lr=['1.667e-03']
[2024-02-10 18:30:55,858 82437:140704363290240][log.py:93 todd.EpochBasedTrainer.lr_schedule_callback after_run_iter] INFO: Iter [2/10] batch={'x': tensor([4, 1]), 'y': tensor([8, 2])} weight=0.004 loss=4.990 lr=['1.667e-03']
[2024-02-10 18:30:55,859 82437:140704363290240][log.py:99 todd.EpochBasedTrainer.lr_schedule_callback before_run_epoch] INFO: Epoch [2/5]
[2024-02-10 18:30:55,861 82437:140704363290240][log.py:93 todd.EpochBasedTrainer.lr_schedule_callback after

### Learning Rate Scale

In [20]:
config = todd.Config(
    type="IterBasedTrainer",
    name="lr_scale_callback",
    dataloader=dict(
        batch_size=2,
        shuffle=True,
        dataset=dict(type="RunnerDataset", n=10),
    ),
    strategy=dict(type="BaseStrategy", model=dict(type="RunnerModel")),
    callbacks=[
        dict(type="OptimizeCallback"),
        dict(
            type="LRScaleCallback",
            lr_scaler=dict(base_batch_size=1),
        ),
        dict(type="LogCallback", interval=1),
    ],
    optimizer=dict(type="SGD", lr=0.005),
    iters=8,
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = todd.RunnerRegistry.build(
        config,
        work_dir=dict(root=work_dirs),
    )
    runner.run()

[2024-02-10 18:30:55,894 82437:140704363290240][lr.py:93 todd.IterBasedTrainer.lr_scale_callback _scale_lr] INFO: base_batch_size=1 batch_size=2 lr_scaler=2.000
[2024-02-10 18:30:55,895 82437:140704363290240][base.py:54 todd.IterBasedTrainer.lr_scale_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-02-10 18:30:55,898 82437:140704363290240][log.py:93 todd.IterBasedTrainer.lr_scale_callback after_run_iter] INFO: Iter [1/8] batch={'x': tensor([3, 2]), 'y': tensor([6, 4])} weight=0.000 loss=5.000
[2024-02-10 18:30:55,900 82437:140704363290240][log.py:93 todd.IterBasedTrainer.lr_scale_callback after_run_iter] INFO: Iter [2/8] batch={'x': tensor([7, 4]), 'y': tensor([14,  8])} weight=0.025 loss=10.863
[2024-02-10 18:30:55,902 82437:140704363290240][log.py:93 todd.IterBasedTrainer.lr_scale_callback after_run_iter] INFO: Iter [3/8] batch={'x': tensor([9, 5]), 'y': tensor([18, 10])} weight=0.080 loss=13.440
[2024-02-10 18:30:55,904 82437:140704363290240][log.py:93 to

### Checkpoint

In [21]:
config = todd.Config(
    type="IterBasedTrainer",
    name="checkpoint_callback",
    dataloader=dict(
        batch_size=2,
        shuffle=True,
        dataset=dict(type="RunnerDataset", n=10),
    ),
    strategy=dict(type="BaseStrategy", model=dict(type="RunnerModel")),
    callbacks=[
        dict(type='OptimizeCallback'),
        dict(type='LogCallback', interval=1),
        dict(type="CheckpointCallback", interval=1),
    ],
    optimizer=dict(type="SGD", lr=0.005),
    iters=8,
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = \
        todd.RunnerRegistry.build(config, work_dir=dict(root=work_dirs))
    runner.run()

    !echo
    !tree {work_dirs}
    !echo

    iter_5 = pathlib.Path(work_dirs) / 'checkpoint_callback' / 'checkpoints' / 'iter_5'
    for f in iter_5.glob('*.pth'):
        print(f'{f.name}:')
        pprint(torch.load(f, 'cpu'))
        print()

    runner: todd.runners.BaseRunner = \
        todd.RunnerRegistry.build(
            config,
            work_dir=dict(root=work_dirs),
            load_from=str(iter_5),
        )
    runner.run()


[2024-02-10 18:30:55,939 82437:140704363290240][base.py:54 todd.IterBasedTrainer.checkpoint_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-02-10 18:30:55,942 82437:140704363290240][log.py:93 todd.IterBasedTrainer.checkpoint_callback after_run_iter] INFO: Iter [1/8] batch={'x': tensor([7, 2]), 'y': tensor([14,  4])} weight=0.000 loss=9.000
[2024-02-10 18:30:55,943 82437:140704363290240][checkpoint.py:80 todd.IterBasedTrainer.checkpoint_callback _save] INFO: Saving state dict to /var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmp_5cambej/checkpoint_callback/checkpoints/iter_1
[2024-02-10 18:30:55,950 82437:140704363290240][log.py:93 todd.IterBasedTrainer.checkpoint_callback after_run_iter] INFO: Iter [2/8] batch={'x': tensor([4, 1]), 'y': tensor([8, 2])} weight=0.022 loss=4.944
[2024-02-10 18:30:55,951 82437:140704363290240][checkpoint.py:80 todd.IterBasedTrainer.checkpoint_callback _save] INFO: Saving state dict to /var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80


/var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmp_5cambej
└── checkpoint_callback
    └── checkpoints
        ├── iter_1
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        ├── iter_2
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        ├── iter_3
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        ├── iter_4
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        ├── iter_5
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        ├── iter_6
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── opt

[2024-02-10 18:30:56,437 82437:140704363290240][checkpoint.py:54 todd.IterBasedTrainer.checkpoint_callback init] INFO: Loading from /var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmp_5cambej/checkpoint_callback/checkpoints/iter_5
[2024-02-10 18:30:56,440 82437:140704363290240][base.py:82 todd.IterBasedTrainer.checkpoint_callback load_model_state_dict] INFO: <All keys matched successfully>
[2024-02-10 18:30:56,441 82437:140704363290240][base.py:54 todd.IterBasedTrainer.checkpoint_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-02-10 18:30:56,444 82437:140704363290240][log.py:93 todd.IterBasedTrainer.checkpoint_callback after_run_iter] INFO: Iter [6/8] batch={'x': tensor([1, 3]), 'y': tensor([2, 6])} weight=0.137 loss=3.725
[2024-02-10 18:30:56,445 82437:140704363290240][checkpoint.py:80 todd.IterBasedTrainer.checkpoint_callback _save] INFO: Saving state dict to /var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmp_5cambej/checkpoint_callback/checkpoints/ite

strategy.pth:
{}

optim.pth:
{'param_groups': [{'dampening': 0,
                   'differentiable': False,
                   'foreach': None,
                   'lr': 0.005,
                   'maximize': False,
                   'momentum': 0,
                   'nesterov': False,
                   'params': [0],
                   'weight_decay': 0}],
 'state': {0: {'momentum_buffer': None}}}

meta.pth:
{'iter_': 5}

model.pth:
OrderedDict([('_weight', tensor(0.1375))])

callbacks.pth:
{'callbacks': [{}, {}, {}]}



In [22]:
config = todd.Config(
    type="EpochBasedTrainer",
    name="checkpoint_callback",
    dataloader=dict(
        batch_size=2,
        shuffle=True,
        dataset=dict(type="RunnerDataset", n=10),
    ),
    strategy=dict(type="BaseStrategy", model=dict(type="RunnerModel")),
    callbacks=[
        dict(type='OptimizeCallback'),
        dict(type='LogCallback', interval=1),
        dict(type="CheckpointCallback", interval=2),
    ],
    optimizer=dict(type="SGD", lr=0.005),
    epochs=3,
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = \
        todd.RunnerRegistry.build(config, work_dir=dict(root=work_dirs))
    runner.run()

    !echo
    !tree {work_dirs}
    !echo

    iter_8 = pathlib.Path(work_dirs) / 'checkpoint_callback' / 'checkpoints' / 'iter_8'
    runner: todd.runners.BaseRunner = \
        todd.RunnerRegistry.build(
            config,
            work_dir=dict(root=work_dirs),
            load_from=str(iter_8),
        )
    runner.run()

    !echo
    !echo {'-' * 20}
    !echo

    iter_10 = pathlib.Path(work_dirs) / 'checkpoint_callback' / 'checkpoints' / 'iter_10'
    runner: todd.runners.BaseRunner = \
        todd.RunnerRegistry.build(
            config,
            work_dir=dict(root=work_dirs),
            load_from=str(iter_10),
        )
    runner.run()


[2024-02-10 18:30:56,499 82437:140704363290240][base.py:54 todd.EpochBasedTrainer.checkpoint_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-02-10 18:30:56,501 82437:140704363290240][log.py:99 todd.EpochBasedTrainer.checkpoint_callback before_run_epoch] INFO: Epoch [1/3]
[2024-02-10 18:30:56,503 82437:140704363290240][log.py:93 todd.EpochBasedTrainer.checkpoint_callback after_run_iter] INFO: Iter [1/15] batch={'x': tensor([2, 9]), 'y': tensor([ 4, 18])} weight=0.000 loss=11.000
[2024-02-10 18:30:56,505 82437:140704363290240][log.py:93 todd.EpochBasedTrainer.checkpoint_callback after_run_iter] INFO: Iter [2/15] batch={'x': tensor([1, 7]), 'y': tensor([ 2, 14])} weight=0.027 loss=7.890
[2024-02-10 18:30:56,506 82437:140704363290240][checkpoint.py:80 todd.EpochBasedTrainer.checkpoint_callback _save] INFO: Saving state dict to /var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmphnnfpp67/checkpoint_callback/checkpoints/iter_2
[2024-02-10 18:30:56,510 82437:14070


/var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmphnnfpp67
└── checkpoint_callback
    └── checkpoints
        ├── iter_10
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        ├── iter_12
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        ├── iter_14
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        ├── iter_2
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        ├── iter_4
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        ├── iter_6
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── 

[2024-02-10 18:30:56,993 82437:140704363290240][checkpoint.py:54 todd.EpochBasedTrainer.checkpoint_callback init] INFO: Loading from /var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmphnnfpp67/checkpoint_callback/checkpoints/iter_8
[2024-02-10 18:30:56,997 82437:140704363290240][base.py:82 todd.EpochBasedTrainer.checkpoint_callback load_model_state_dict] INFO: <All keys matched successfully>
[2024-02-10 18:30:56,997 82437:140704363290240][base.py:54 todd.EpochBasedTrainer.checkpoint_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-02-10 18:30:56,999 82437:140704363290240][log.py:99 todd.EpochBasedTrainer.checkpoint_callback before_run_epoch] INFO: Epoch [2/3]
[2024-02-10 18:30:57,002 82437:140704363290240][log.py:93 todd.EpochBasedTrainer.checkpoint_callback after_run_iter] INFO: Iter [9/15] batch={'x': tensor([2, 8]), 'y': tensor([ 4, 16])} weight=0.237 loss=8.812
[2024-02-10 18:30:57,004 82437:140704363290240][log.py:93 todd.EpochBasedTrainer.checkpoint_c


--------------------



[2024-02-10 18:30:57,446 82437:140704363290240][checkpoint.py:54 todd.EpochBasedTrainer.checkpoint_callback init] INFO: Loading from /var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmphnnfpp67/checkpoint_callback/checkpoints/iter_10
[2024-02-10 18:30:57,450 82437:140704363290240][base.py:82 todd.EpochBasedTrainer.checkpoint_callback load_model_state_dict] INFO: <All keys matched successfully>
[2024-02-10 18:30:57,451 82437:140704363290240][base.py:54 todd.EpochBasedTrainer.checkpoint_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-02-10 18:30:57,452 82437:140704363290240][log.py:99 todd.EpochBasedTrainer.checkpoint_callback before_run_epoch] INFO: Epoch [3/3]
[2024-02-10 18:30:57,456 82437:140704363290240][log.py:93 todd.EpochBasedTrainer.checkpoint_callback after_run_iter] INFO: Iter [11/15] batch={'x': tensor([ 2, 10]), 'y': tensor([ 4, 20])} weight=0.295 loss=10.230
[2024-02-10 18:30:57,458 82437:140704363290240][log.py:93 todd.EpochBasedTrainer.checkpo

In [23]:
config = todd.Config(
    type="EpochBasedTrainer",
    name="checkpoint_callback",
    dataloader=dict(
        batch_size=2,
        shuffle=True,
        dataset=dict(type="RunnerDataset", n=10),
    ),
    strategy=dict(type="BaseStrategy", model=dict(type="RunnerModel")),
    callbacks=[
        dict(type='OptimizeCallback'),
        dict(type='LogCallback', interval=1),
        dict(type="CheckpointCallback", interval=1, by_epoch=True),
    ],
    optimizer=dict(type="SGD", lr=0.005),
    epochs=3,
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = \
        todd.RunnerRegistry.build(config, work_dir=dict(root=work_dirs))
    runner.run()

    !echo
    !tree {work_dirs}
    !echo

    epoch_2 = pathlib.Path(work_dirs) / 'checkpoint_callback' / 'checkpoints' / 'epoch_2'
    runner: todd.runners.BaseRunner = \
        todd.RunnerRegistry.build(
            config,
            work_dir=dict(root=work_dirs),
            load_from=str(epoch_2),
        )
    runner.run()


[2024-02-10 18:30:57,501 82437:140704363290240][base.py:54 todd.EpochBasedTrainer.checkpoint_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-02-10 18:30:57,502 82437:140704363290240][log.py:99 todd.EpochBasedTrainer.checkpoint_callback before_run_epoch] INFO: Epoch [1/3]
[2024-02-10 18:30:57,504 82437:140704363290240][log.py:93 todd.EpochBasedTrainer.checkpoint_callback after_run_iter] INFO: Iter [1/15] batch={'x': tensor([5, 6]), 'y': tensor([10, 12])} weight=0.000 loss=11.000
[2024-02-10 18:30:57,507 82437:140704363290240][log.py:93 todd.EpochBasedTrainer.checkpoint_callback after_run_iter] INFO: Iter [2/15] batch={'x': tensor([10,  7]), 'y': tensor([20, 14])} weight=0.027 loss=16.766
[2024-02-10 18:30:57,509 82437:140704363290240][log.py:93 todd.EpochBasedTrainer.checkpoint_callback after_run_iter] INFO: Iter [3/15] batch={'x': tensor([3, 1]), 'y': tensor([6, 2])} weight=0.070 loss=3.860
[2024-02-10 18:30:57,511 82437:140704363290240][log.py:93 todd.Epoc


/var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmpophxxzac
└── checkpoint_callback
    └── checkpoints
        ├── epoch_1
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        ├── epoch_2
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        ├── epoch_3
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        └── latest -> epoch_3

7 directories, 15 files



[2024-02-10 18:30:58,007 82437:140704363290240][checkpoint.py:54 todd.EpochBasedTrainer.checkpoint_callback init] INFO: Loading from /var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmpophxxzac/checkpoint_callback/checkpoints/epoch_2
[2024-02-10 18:30:58,011 82437:140704363290240][base.py:82 todd.EpochBasedTrainer.checkpoint_callback load_model_state_dict] INFO: <All keys matched successfully>
[2024-02-10 18:30:58,012 82437:140704363290240][base.py:54 todd.EpochBasedTrainer.checkpoint_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-02-10 18:30:58,013 82437:140704363290240][log.py:99 todd.EpochBasedTrainer.checkpoint_callback before_run_epoch] INFO: Epoch [3/3]
[2024-02-10 18:30:58,017 82437:140704363290240][log.py:93 todd.EpochBasedTrainer.checkpoint_callback after_run_iter] INFO: Iter [11/15] batch={'x': tensor([1, 8]), 'y': tensor([ 2, 16])} weight=0.275 loss=7.763
[2024-02-10 18:30:58,019 82437:140704363290240][log.py:93 todd.EpochBasedTrainer.checkpoint

### Monitor

In [24]:
class CustomError(RuntimeError):
    pass

In [25]:
@todd.RunnerRegistry.register_()
class FaultyValidator(todd.runners.Validator):

    def _run_iter(self, *args, **kwargs) -> NoReturn:
        raise CustomError("faulty runner")

In [26]:
config = todd.Config(
    type='FaultyValidator',
    name='monitor_callback',
    dataloader=dict(batch_size=1, dataset=dict(type='RunnerDataset', n=20)),
    strategy=dict(type='BaseStrategy', model=dict(type='RunnerModel')),
    callbacks=[
        dict(type='MonitorCallback'),
        dict(type='LogCallback', interval=5, with_file_handler=True),
    ],
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = \
        todd.RunnerRegistry.build(config, work_dir=dict(root=work_dirs))
    try:
        runner.run()
    except CustomError as e:
        pass

    !echo
    !cat {work_dirs}/monitor_callback/*.log


[2024-02-10 18:30:58,070 82437:140704363290240][base.py:54 todd.FaultyValidator.monitor_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-02-10 18:30:58,074 82437:140704363290240][monitor.py:26 todd.FaultyValidator.monitor_callback __exit__] ERROR: Unable to run iter_=1
batch={'x': tensor([1]), 'y': tensor([2])}
memo={'dataloader': <torch.utils.data.dataloader._SingleProcessDataLoaderIter object at 0x111620910>}
Traceback (most recent call last):
  File "/Users/bytedance/.local/share/virtualenvs/todd-ARrcnwyq/lib/python3.11/site-packages/todd/runners/base.py", line 222, in _run
    memo = self._run_iter(batch, memo)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/ipykernel_82437/1715875531.py", line 5, in _run_iter
    raise CustomError("faulty runner")
CustomError: faulty runner



[2024-02-10 18:30:58,070 82437:140704363290240][base.py:54 todd.FaultyValidator.monitor_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-02-10 18:30:58,074 82437:140704363290240][monitor.py:26 todd.FaultyValidator.monitor_callback __exit__] ERROR: Unable to run iter_=1
batch={'x': tensor([1]), 'y': tensor([2])}
memo={'dataloader': <torch.utils.data.dataloader._SingleProcessDataLoaderIter object at 0x111620910>}
Traceback (most recent call last):
  File "/Users/bytedance/.local/share/virtualenvs/todd-ARrcnwyq/lib/python3.11/site-packages/todd/runners/base.py", line 222, in _run
    memo = self._run_iter(batch, memo)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/ipykernel_82437/1715875531.py", line 5, in _run_iter
    raise CustomError("faulty runner")
CustomError: faulty runner


### Priorities

## Strategies

In [27]:
config = todd.Config(
    type="EpochBasedTrainer",
    name="strategy_load_model_from",
    dataloader=dict(
        batch_size=2,
        shuffle=True,
        dataset=dict(type="RunnerDataset", n=10),
    ),
    strategy=dict(type="BaseStrategy", model=dict(type="RunnerModel")),
    callbacks=[
        dict(type='OptimizeCallback'),
        dict(type='LogCallback', interval=1),
        dict(type="CheckpointCallback", interval=1, by_epoch=True),
    ],
    optimizer=dict(type="SGD", lr=0.005),
    epochs=3,
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = \
        todd.RunnerRegistry.build(config, work_dir=dict(root=work_dirs))
    runner.run()

    !echo
    !echo {'-' * 20}
    !echo

    epoch_2 = (pathlib.Path(work_dirs) / 'strategy_load_model_from' / 'checkpoints' / 'epoch_2' / 'model.pth')
    runner: todd.runners.BaseRunner = \
        todd.RunnerRegistry.build(config, work_dir=dict(root=work_dirs))
    runner.strategy.load_model_from(epoch_2)
    runner.run()


[2024-02-10 18:30:58,396 82437:140704363290240][base.py:54 todd.EpochBasedTrainer.strategy_load_model_from __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-02-10 18:30:58,398 82437:140704363290240][log.py:99 todd.EpochBasedTrainer.strategy_load_model_from before_run_epoch] INFO: Epoch [1/3]
[2024-02-10 18:30:58,400 82437:140704363290240][log.py:93 todd.EpochBasedTrainer.strategy_load_model_from after_run_iter] INFO: Iter [1/15] batch={'x': tensor([9, 6]), 'y': tensor([18, 12])} weight=0.000 loss=15.000
[2024-02-10 18:30:58,402 82437:140704363290240][log.py:93 todd.EpochBasedTrainer.strategy_load_model_from after_run_iter] INFO: Iter [2/15] batch={'x': tensor([10,  2]), 'y': tensor([20,  4])} weight=0.037 loss=11.775
[2024-02-10 18:30:58,405 82437:140704363290240][log.py:93 todd.EpochBasedTrainer.strategy_load_model_from after_run_iter] INFO: Iter [3/15] batch={'x': tensor([3, 7]), 'y': tensor([ 6, 14])} weight=0.067 loss=9.663
[2024-02-10 18:30:58,407 82437:140704363


--------------------



[2024-02-10 18:30:58,873 82437:140704363290240][base.py:54 todd.EpochBasedTrainer.strategy_load_model_from __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-02-10 18:30:58,875 82437:140704363290240][base.py:97 todd.EpochBasedTrainer.strategy_load_model_from load_model_from] INFO: Loading model from /var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmp_kof8sjl/strategy_load_model_from/checkpoints/epoch_2/model.pth
[2024-02-10 18:30:58,877 82437:140704363290240][base.py:82 todd.EpochBasedTrainer.strategy_load_model_from load_model_state_dict] INFO: <All keys matched successfully>
[2024-02-10 18:30:58,878 82437:140704363290240][log.py:99 todd.EpochBasedTrainer.strategy_load_model_from before_run_epoch] INFO: Epoch [1/3]
[2024-02-10 18:30:58,882 82437:140704363290240][log.py:93 todd.EpochBasedTrainer.strategy_load_model_from after_run_iter] INFO: Iter [1/15] batch={'x': tensor([ 1, 10]), 'y': tensor([ 2, 20])} weight=0.275 loss=9.488
[2024-02-10 18:30:58,884 82437:14070436

## Dry Run

In [28]:
todd.Store.DRY_RUN = True